# Imports

In [2]:
import gensim.downloader as api
import json
import os
import numpy as np

from sklearn.decomposition import PCA

In [10]:
import gensim
gensim.__version__  # I use '3.6.0'

'3.6.0'

In [3]:
def load_legacy_w2v(w2v_file, dim=50):
    vectors = {}
    with open(w2v_file, 'r') as f:
        for line in f:
            vect = line.strip().rsplit()
            word = vect[0]
            vect = np.array([float(x) for x in vect[1:]])
            if(dim == len(vect)):
                vectors[word] = vect
        
    return vectors, dim

# Train Word2Vec

In [4]:
# Pretrained embeddings from the l2 reddit corpus
# https://github.com/TManzini/DebiasMulticlassWordEmbedding
reddit_path = os.path.join('data', 'data_vocab_race_pre_trained.w2v')

# Debias

Two steps:
1. Identify the bias subspace
2. remove this component from the set of embeddings

In [5]:
def pruneWordVecs(wordVecs):
    newWordVecs = {}
    for word, vec in wordVecs.items():
        valid=True
        if(not all([c.isalpha() for c in word])):
            valid = False
        if(valid):
            newWordVecs[word] = vec
    return newWordVecs


# Load unbiased embeddings
reddit_embeddings, embedding_dim = load_legacy_w2v(reddit_path, 50)

# take out words that do not only contain letters
word_vectors = pruneWordVecs(reddit_embeddings)
print("Remaining words after prune:", len(word_vectors))

# Get defining sets
def_sets_path = os.path.join('data', 'vocab', 'race_attributes_optm.json')
with open(def_sets_path, 'r') as f:
    def_sets =json.load(f)
def_sets = {i: v for i, v in enumerate(def_sets["definite_sets"])}

# Calculate the mean for each defining set
means = {}
for k, v in def_sets.items():
    wSet = []
    for word in v:
        wSet.append(word_vectors[word])
    set_vectors = np.array(wSet)
    means[k] = np.mean(set_vectors, axis=0)

# Subtract each word by the mean of the set
matrix = []
for k, v in def_sets.items():
    wSet = []
    for word in v:
        wSet.append(word_vectors[word])
    set_vectors = np.array(wSet)
    diffs = set_vectors - means[k]
    matrix.append(diffs)
matrix = np.concatenate(matrix)

# PCA
k = 2
pca = PCA(n_components=k)
pca.fit(matrix)
subspace = pca.components_

# ---------------  Remove the bias subspace -------------------

# Get neutral words
attribute_path = os.path.join('data', 'vocab', 'race_attributes_optm.json')
with open(attribute_path, "r") as f:	
    attributes = json.load(f)
    roles = attributes["analogy_templates"]['role']

neutral_words = []
for value in roles.values():
    neutral_words.extend(value)

new_vocab = word_vectors.copy()

# Neutralize
for word in neutral_words:
    if word in word_vectors:
        embedding = word_vectors[word]

        # Project neutral word onto bias subspace
        v_b = np.zeros_like(embedding)
        for component in subspace:
            v_b += np.dot(embedding.transpose(), component) * component

        # Remove bias subspace and normalize to unit vector
        new_v = (embedding - v_b) / np.linalg.norm(embedding - v_b)
        new_vocab[word] = new_v

    # Normalize ALL words, not just the neurtral words chosen
    for word, embedding in new_vocab.items():
        new_vocab[word] = embedding / np.linalg.norm(embedding)

# Equalize
for eq_set in def_sets.values():
    mean = np.zeros(matrix.shape[1])

    for word in eq_set:
        # Calculate the mean of the words in the set
        mean += new_vocab[word]
        mean /= float(len(eq_set))

        # Project this mean onto the bias subspace
        mean_b = np.zeros_like(mean)
        for component in subspace:
            mean_b += np.dot(mean.transpose(), component) * component

        # Remove the bias subspace from the mean
        upsilon = mean - mean_b

    for word in eq_set:
        embedding = new_vocab[word]

        # project the word from the equality set onto the bias subspace
        v_b = np.zeros_like(embedding)
        for component in subspace:
            v_b += np.dot(embedding.transpose(), component) * component

        # centering? Subtract the word projected onto the bia subspace by the mean projected onto the bias subspace
        frac = (v_b - mean_b) / np.linalg.norm(v_b - mean_b)

        # why? equalize?
        new_v = upsilon + np.sqrt(1 - np.sum(np.square(upsilon))) * frac

        # Update with the new embedding
        new_vocab[word] = new_v

Remaining words after prune: 44895


# Double Hard

In [6]:
# Double Hard
from sklearn.cluster import AgglomerativeClustering, KMeans

def calculate_main_pca_components(word_vectors):
    """From https://github.com/uvavision/Double-Hard-Debias/blob/master/GloVe_Debias.ipynb"""
    vectors = word_vectors.values() # word_vectors.vectors
    wv_mean = np.mean(np.array(vectors), axis=0)
    wv_hat = vectors - wv_mean
    main_pca = PCA()
    main_pca.fit(wv_hat)
    return main_pca

def neutralize_and_equalize_with_frequency_removal(vocab, words, eq_sets, bias_subspace, embedding_dim, principal_component):
    """
    vocab - dictionary mapping words to embeddings
    words - words to neutralize
    eq_sets - set of equality sets
    bias_subspace - subspace of bias from identify_bias_subspace
    embedding_dim - dimensions of the word embeddings
    """

    if bias_subspace.ndim == 1:
        bias_subspace = np.expand_dims(bias_subspace, 0)
    elif bias_subspace.ndim != 2:
        raise ValueError("bias subspace should be either a matrix or vector")

    full_set = set(list(words) + [word for eq_words in eq_sets for word in eq_words])
    freq_vocab = vocab.copy()

    for word in full_set:
        vector = freq_vocab[word]
        projection = np.dot(np.dot(np.transpose(principal_component), vector), principal_component)
        freq_vocab[word] = vector - projection

    return neutralize_and_equalize(freq_vocab, words, eq_sets, bias_subspace, embedding_dim)


def neutralize_and_equalize(vocab, words, eq_sets, bias_subspace, embedding_dim):
    """
    vocab - dictionary mapping words to embeddings
    words - words to neutralize
    eq_sets - set of equality sets
    bias_subspace - subspace of bias from identify_bias_subspace
    embedding_dim - dimensions of the word embeddings
    """

    if bias_subspace.ndim == 1:
        bias_subspace = np.expand_dims(bias_subspace, 0)
    elif bias_subspace.ndim != 2:
        raise ValueError("bias subspace should be either a matrix or vector")

    new_vocab = vocab.copy()
    for w in words:
        # get projection onto bias subspace
        if w in vocab:
            v = vocab[w]
            v_b = project_onto_subspace(v, bias_subspace)

            new_v = (v - v_b) / np.linalg.norm(v - v_b)
            #print np.linalg.norm(new_v)
            # update embedding
            new_vocab[w] = new_v

    normalize(new_vocab)

    for eq_set in eq_sets:
        mean = np.zeros((embedding_dim,))

        #Make sure the elements in the eq sets are valid
        cleanEqSet = []
        for w in eq_set:
            try:
                _ = new_vocab[w]
                cleanEqSet.append(w)
            except KeyError as e:
                pass

        for w in cleanEqSet:
            mean += new_vocab[w]
        mean /= float(len(cleanEqSet))

        mean_b = project_onto_subspace(mean, bias_subspace)
        upsilon = mean - mean_b

        for w in cleanEqSet:
            v = new_vocab[w]
            v_b = project_onto_subspace(v, bias_subspace)

            frac = (v_b - mean_b) / np.linalg.norm(v_b - mean_b)
            new_v = upsilon + np.sqrt(1 - np.sum(np.square(upsilon))) * frac

            new_vocab[w] = new_v

    return new_vocab

def project_onto_subspace(vector, subspace):
    v_b = np.zeros_like(vector)
    for component in subspace:
        v_b += np.dot(vector.transpose(), component) * component
    return v_b

def normalize(word_vectors):
    for k, v in word_vectors.items():
        word_vectors[k] = v / np.linalg.norm(v)

def cluster(X1, random_state, y_true, num=2):
    kmeans_1 = KMeans(n_clusters=num, random_state=random_state).fit(X1)
    y_pred_1 = kmeans_1.predict(X1)
    correct = [1 if item1 == item2 else 0 for (item1,item2) in zip(y_true, y_pred_1) ]
    return max(sum(correct)/float(len(correct)), 1 - sum(correct)/float(len(correct)))

def get_most_biased(word_vectors, subspace, n_biased=500):
    """
    Get vectors with most positive and negative bias wrt subspace.
    """

    biases = {}

    for word, vector in word_vectors.items():
        # 1d case
        bias = np.dot(vector, subspace)
        biases[word] = bias

    sorted_biases = sorted(list(biases.items()), key=lambda x: x[1], reverse=True)

    positive_bias = sorted_biases[:n_biased]
    negative_bias = list(reversed(sorted_biases[-n_biased:]))

    return positive_bias, negative_bias


def identify_bias_subspace(vocab, def_sets, subspace_dim, embedding_dim):
    """
    Similar to bolukbasi's implementation at
    https://github.com/tolga-b/debiaswe/blob/master/debiaswe/debias.py
    vocab - dictionary mapping words to embeddings
    def_sets - sets of words that represent extremes? of the subspace
            we're interested in (e.g. man-woman, boy-girl, etc. for binary gender)
    subspace_dim - number of vectors defining the subspace
    embedding_dim - dimensions of the word embeddings
    """
    # calculate means of defining sets
    means = {}
    for k, v in def_sets.items():
        wSet = []
        for w in v:
            try:
                wSet.append(vocab[w])
            except KeyError as e:
                pass
        set_vectors = np.array(wSet)
        means[k] = np.mean(set_vectors, axis=0)

    # calculate vectors to perform PCA
    matrix = []
    for k, v in def_sets.items():
        wSet = []
        for w in v:
            try:
                wSet.append(vocab[w])
            except KeyError as e:
                pass
        set_vectors = np.array(wSet)
        diffs = set_vectors - means[k]
        matrix.append(diffs)

    matrix = np.concatenate(matrix)

    pca = PCA(n_components=subspace_dim)
    pca.fit(matrix)

    return pca.components_

In [7]:
subspace = identify_bias_subspace(word_vectors, def_sets, 2, 50)

# get all positive and negative words words within the subspace (moving towards or away from some race)
positive_words, negative_words = get_most_biased(word_vectors, subspace[0], 1000)

# get all biased words (minus those in the defining/equality set)
positive_words = [word for word, sim in positive_words]
negative_words = [word for word, sim in negative_words]

eq_sets = [set for _, set in def_sets.items()]
biased_words = set(positive_words + negative_words) - set(np.array(eq_sets).flatten())

# 1 for more positive biased words, 0 for more negative biased words
y_true = [ 1 if word in positive_words else 0 for word in biased_words]

In [8]:
# calculate decentralized embedding pca
summation = np.zeros(shape=50)
for word in word_vectors:
    summation += word_vectors[word]
wv_mean = summation/len(word_vectors)

wv_hat = []
for word in word_vectors:
    wv_hat.append(word_vectors[word] - wv_mean)

main_pca = PCA()
main_pca.fit(wv_hat)
main_pca = main_pca.components_

# run debias for each principal component and capture best precision
precisions = []
debiases = []

for pc in main_pca:
    debiased_frequency = neutralize_and_equalize_with_frequency_removal(word_vectors, biased_words, eq_sets, subspace, 50, pc)
    x_dhd = [debiased_frequency[word] for word in biased_words]    
    precisions.append(cluster(X1=np.array(x_dhd), random_state=3, y_true=y_true, num=2))
    debiases.append(debiased_frequency)

# use debias with the lowest precision for further study
db = debiases[np.argmin(precisions)]

# Evaluate

In [9]:
from gensim.models.keyedvectors import Word2VecKeyedVectors
from scipy import spatial

def generateAnalogies(analogyTemplates, keyedVecs):

	# generates anaology templates from passing stereotypical roles
	# Ex. [['caucasian', 'manager', 'asian'], ['black', 'musician', 'caucasian'], ['asian', 'teacher', 'black']]
	expandedAnalogyTemplates = []
	for A, stereotypes in analogyTemplates.items():
		for B, _ in analogyTemplates.items():
			if(A != B):
				for stereotype in stereotypes:
					expandedAnalogyTemplates.append([A, stereotype, B])

	# Test every analogy template with every word in our vocab
	analogies = []
	outputGroups = []
	for a,b,x in expandedAnalogyTemplates:
		outputs = scoredAnalogyAnswers(a,b,x,keyedVecs)
		formattedOutput = []
		
		for score, a_w, b_w, x_w, y_w in outputs:
			
			analogy = str(a_w) + " is to " + str(b_w) + " as " + str(x_w) + " is to " + str(y_w)
			analogyRaw = [a_w, b_w, x_w, y_w]
			analogies.append([score, analogy, analogyRaw])
			formattedOutput.append([score, analogy, analogyRaw])
		outputGroups.append(formattedOutput)

	analogies = sorted(analogies, key=lambda x:-x[0])
	return analogies, outputGroups

def convert_legacy_to_keyvec(legacy_w2v):
    dim = len(list(word_vectors.values())[0])
    vectors = Word2VecKeyedVectors(dim)

    ws = []
    vs = []

    for word, vect in legacy_w2v.items():
        ws.append(word)
        vs.append(vect)
        assert(len(vect) == dim)
    vectors.add(ws, vs, replace=True)
    return vectors

def scoredAnalogyAnswers(a,b,x, keyedVecs, thresh=12.5):
	"""
	cosine similarity score of the analogies
	closer to 1 = the difference between a and b is very similar to
	the distance between x and y
	Ex. 'apple is to fruit as carrot is to vegetable
	the vector difference between apple and fruit
	is compared to the difference in carrot and vegetable
	if they are good analogies then the cosine similarities will
	be 1
	"""

	words = [w for w in keyedVecs.vocab if np.linalg.norm(np.array(keyedVecs[w])-np.array(keyedVecs[x])) < thresh]
	
	def cos(a,b,x,y):
		aVec = np.array(keyedVecs[a])
		bVec = np.array(keyedVecs[b])
		xVec = np.array(keyedVecs[x])
		yVec = np.array(keyedVecs[y])
		numerator = (aVec-bVec).dot(xVec-yVec)
		denominator = np.linalg.norm(aVec-bVec)*np.linalg.norm(xVec-yVec)
		return numerator/(denominator if denominator != 0 else 1e-6)

	return sorted([(cos(a,b,x,y), a,b,x,y) for y in words], reverse=True)
 
def multiclass_evaluation(embeddings, targets, attributes):
	targets_eval = []
	for targetSet in targets:
		for target in targetSet:
			for attributeSet in attributes:
				targets_eval.append(_unary_s(embeddings, target, attributeSet))
	m_score = np.mean(targets_eval)
	return m_score, targets_eval

def _unary_s(embeddings, target, attributes):
	return np.mean([ spatial.distance.cosine(embeddings[target], embeddings[ai]) for ai in attributes ])

In [11]:
biasedAnalogies, biasedAnalogyGroups = generateAnalogies(roles, convert_legacy_to_keyvec(word_vectors))
hardDebiasedAnalogies, hardDebiasedAnalogyGroups = generateAnalogies(roles, convert_legacy_to_keyvec(new_vocab))
doubleHardDebiasedAnalogies, doubleHardDebiasedAnalogyGroups = generateAnalogies(roles, convert_legacy_to_keyvec(db))

In [12]:
limit = 20
print("Biased Analogies (0-" + str(limit) + ")")
for score, analogy, _ in biasedAnalogies[:limit]:
    print(score, analogy)

print("="*20, "\n\n")
print("Hard Debiased Analogies (0-" + str(limit) + ")")
for score, analogy, _ in hardDebiasedAnalogies[:limit]:
    print(score, analogy)

print("="*20, "\n\n")
print("Double-Hard Debiased Analogies (0-" + str(limit) + ")")
for score, analogy, _ in doubleHardDebiasedAnalogies[:limit]:
    print(score, analogy)

Biased Analogies (0-20)
0.7440956387393549 asian is to laborer as caucasian is to maxim
0.7242655048287052 asian is to laborer as caucasian is to quorum
0.7227820072311668 asian is to engineer as caucasian is to sociologist
0.7215993085150522 black is to runner as caucasian is to fricative
0.714185409976964 asian is to laborer as caucasian is to forgery
0.7123000623464896 black is to runner as caucasian is to rifleman
0.7109950201019208 black is to runner as caucasian is to alveolar
0.7099742338897985 black is to runner as caucasian is to para
0.7098601539812843 black is to runner as caucasian is to infinitive
0.7093358220823148 asian is to engineer as caucasian is to astronomer
0.7057686454979323 asian is to engineer as caucasian is to statistician
0.7056965451006092 asian is to laborer as caucasian is to moniker
0.705575826351776 caucasian is to executive as asian is to australian
0.7049933109196884 asian is to laborer as caucasian is to predicate
0.7049448993105739 asian is to labor

In [13]:
print("Performing Evaluation")
print("="*20,)

evalTargets, evalAttrs = attributes['eval_targets'], attributes["analogy_templates"]['role'].values()

biasedMAC, biasedDistribution = multiclass_evaluation(word_vectors, evalTargets, evalAttrs)
print("Biased MAC:", biasedMAC)
debiasedMAC, debiasedDistribution = multiclass_evaluation(new_vocab, evalTargets, evalAttrs)
print("HARD MAC:", debiasedMAC)
doubledebiasedMAC, doubledebiasedDistribution = multiclass_evaluation(db, evalTargets, evalAttrs)
print("DOUBLE HARD MAC:", doubledebiasedMAC)

Performing Evaluation
Biased MAC: 0.8920888687350088
HARD MAC: 0.9809887952002936
DOUBLE HARD MAC: 0.9406683954862781


## Concept Categorization

Cluster a set of words into different categorical subsets. Metric is fraction of the total number of the words that are correctly classified

## Concept

In [14]:
import glob
from sklearn.utils import Bunch
from six import iteritems

def evaluate_categorization(word_vectors, X, y, method='kmeans', seed=None):
    """
    Evaluate embeddings on categorization task.
    Parameters
    ----------
    w: dict
      Embeddings to test.
    X: vector, shape: (n_samples, )
      Vector of words.
    y: vector, shape: (n_samples, )
      Vector of cluster assignments.
    """

    # Get all word embeddings into a matrix
    vectors = []
    for word in word_vectors:
        vectors.append(word_vectors[word])

    # Mean of all embeddings
    mean_vector = np.mean(vectors, axis=0, keepdims=True)

    w = word_vectors

    new_x = []
    new_y = []
    exist_cnt = 0
    for idx, word in enumerate(X.flatten()):
        if word in w :
            new_x.append(X[idx])
            new_y.append(y[idx])
            exist_cnt += 1

    # Number of words in BLESS that also exists in our vocabulary
    print('exist {} in {}'.format(exist_cnt, len(X)))

    X = np.array(new_x)
    y = np.array(new_y)

    # Put all the words that were in both BLESS and our vocab into a matrix
    words = np.vstack([w.get(word, mean_vector) for word in X.flatten()])
    ids = np.random.RandomState(seed).choice(range(len(X)), len(X), replace=False)

    # Evaluate clustering on several hyperparameters of AgglomerativeClustering and
    # KMeans
    best_purity = 0

    if method == "all" or method == "agglomerative":
        best_purity = calculate_purity(y[ids], AgglomerativeClustering(n_clusters=len(set(y)),
                                                                       affinity="euclidean",
                                                                       linkage="ward").fit_predict(words[ids]))
        for affinity in ["cosine", "euclidean"]:
            for linkage in ["average", "complete"]:
                purity = calculate_purity(y[ids], AgglomerativeClustering(n_clusters=len(set(y)),
                                                                          affinity=affinity,
                                                                          linkage=linkage).fit_predict(words[ids]))
                best_purity = max(best_purity, purity)

    if method == "all" or method == "kmeans":
        purity = calculate_purity(y[ids], KMeans(random_state=seed, n_init=10, n_clusters=len(set(y))).
                                  fit_predict(words[ids]))
        best_purity = max(purity, best_purity)

    return best_purity


def calculate_purity(y_true, y_pred, verbose=False):
    """
    Calculate purity for given true and predicted cluster labels.
    Parameters
    ----------
    y_true: array, shape: (n_samples, 1)
      True cluster labels
    y_pred: array, shape: (n_samples, 1)
      Cluster assingment.
    Returns
    -------
    purity: float
      Calculated purity.
    """
    assert len(y_true) == len(y_pred)
    true_clusters = np.zeros(shape=(len(set(y_true)), len(y_true)))  # creates sparse array (categories, words both in bench and vocab)
    pred_clusters = np.zeros_like(true_clusters)  # creates sparse array (categories, words both in bench and vocab)
    for id, cl in enumerate(set(y_true)):
        if verbose:
            print("true:", id)
        true_clusters[id] = (y_true == cl).astype("int")  # Everwhere the label is of a certain class, put a 1
    for id, cl in enumerate(set(y_pred)):
        if verbose:
            print("pred:", id)
        pred_clusters[id] = (y_pred == cl).astype("int")  # Everwhere the label is in a certain cluster, put a 1

    # For each clust in the prediction, find the true cluster that has the MOST overlap
    # Sum up the number of words that overlap between the pred and true cluster that has the most overlap
    # Divide this by (the number of words both in bench and vocab)
    M = pred_clusters.dot(true_clusters.T)
    return 1. / len(y_true) * np.sum(np.max(M, axis=1))
    

def evaluate_cate(wv_dict, benchmarks, method='all', seed=None):
    categorization_tasks = benchmarks
    categorization_results = {}

    # Calculate results using helper function
    for name, data in iteritems(categorization_tasks):
        print("Sample data from {}, num of samples: {} : \"{}\" is assigned class {}".format(
            name, len(data.X), data.X[0], data.y[0]))
        categorization_results[name] = evaluate_categorization(wv_dict, data.X, data.y, method=method, seed=None)
        print("Cluster purity on {} {}".format(name, categorization_results[name]))

def create_bunches(bench_paths):
    output = {}
    for path in bench_paths:
        files = glob.glob(os.path.join(path, "*.txt"))
        name = os.path.basename(path)
        if name == 'EN-BATTIG':
            sep = ","
        else:
            sep = " "

        X = []
        y = []
        names = []
        for cluster_id, file_name in enumerate(files):
            with open(file_name) as f:
                lines = f.read().splitlines()[:]
                X += [l.split(sep) for l in lines]
                y += [os.path.basename(file_name).split(".")[0]] * len(lines)
        output[name] = Bunch(X=np.array(X, dtype="object"), y=np.array(y).astype("object"))

        if sep == ",":
            data = output[name]
            output[name] = Bunch(X=data.X[:, 0], y=data.y, freq=data.X[:, 1], frequency=data.X[:, 2], rank=data.X[:, 3], rfreq=data.X[:, 4])

            
    
    return output

In [21]:
bench_names = ["EN-AP", "EN-ESSLI-2c", "EN-ESSLI-2b", "EN-ESSLI-1a", "EN-BATTIG", "EN-BLESS", ]
bench_paths = [os.path.join('data', 'concept_clustering', name) for name in bench_names]
benchmarks = create_bunches(bench_paths)
benchmarks

{'EN-AP': {'X': array([['acne'],
         ['anthrax'],
         ['arthritis'],
         ['asthma'],
         ['cancer'],
         ['cholera'],
         ['cirrhosis'],
         ['diabetes'],
         ['eczema'],
         ['flu'],
         ['glaucoma'],
         ['hepatitis'],
         ['leukemia'],
         ['malnutrition'],
         ['meningitis'],
         ['plague'],
         ['rheumatism'],
         ['smallpox'],
         ['aluminium'],
         ['bismuth'],
         ['cadmium'],
         ['calcium'],
         ['carbon'],
         ['charcoal'],
         ['copper'],
         ['germanium'],
         ['helium'],
         ['hydrogen'],
         ['iron'],
         ['lithium'],
         ['magnesium'],
         ['neon'],
         ['nitrogen'],
         ['oxygen'],
         ['platinum'],
         ['potassium'],
         ['silver'],
         ['titanium'],
         ['zinc'],
         ['compulsion'],
         ['conscience'],
         ['deterrence'],
         ['disincentive'],
         ['dynami

In [22]:
evaluate_cate(word_vectors, benchmarks)
print()
evaluate_cate(new_vocab, benchmarks)
print()
evaluate_cate(db, benchmarks)

Sample data from EN-AP, num of samples: 402 : "['acne']" is assigned class illness
exist 328 in 402
Cluster purity on EN-AP 0.5091463414634146
Sample data from EN-ESSLI-2c, num of samples: 45 : "['acquire']" is assigned class exchange-exchange
exist 45 in 45
Cluster purity on EN-ESSLI-2c 0.5555555555555556
Sample data from EN-ESSLI-2b, num of samples: 40 : "['chicken']" is assigned class HI
exist 40 in 40
Cluster purity on EN-ESSLI-2b 0.8500000000000001
Sample data from EN-ESSLI-1a, num of samples: 44 : "['bottle']" is assigned class tool-artifact
exist 44 in 44
Cluster purity on EN-ESSLI-1a 0.6363636363636364
Sample data from EN-BATTIG, num of samples: 5287 : "word" is assigned class toy
exist 3442 in 5287
Cluster purity on EN-BATTIG 0.3448576409064497
Sample data from EN-BLESS, num of samples: 200 : "['carp']" is assigned class water_animal
exist 193 in 200
Cluster purity on EN-BLESS 0.5181347150259068

Sample data from EN-AP, num of samples: 402 : "['acne']" is assigned class illnes

Control:

Embeddings given to us by the Black is to Criminal Paper.

In [24]:
biased_path = os.path.join('data', 'data_vocab_race_pre_trained.w2v')
biased_embeddings, embedding_dim = load_legacy_w2v(biased_path, 50)
# take out words that do not only contain letters
biased_word_vectors = pruneWordVecs(biased_embeddings)
evaluate_cate(biased_word_vectors, benchmarks)

print()

debiased_path = os.path.join('data','data_vocab_race_hard_debias.w2v')
debiased_embeddings, embedding_dim = load_legacy_w2v(debiased_path, 50)
# take out words that do not only contain letters
debiased_word_vectors = pruneWordVecs(debiased_embeddings)
evaluate_cate(debiased_word_vectors, benchmarks)

Sample data from EN-AP, num of samples: 402 : "['acne']" is assigned class illness
exist 328 in 402
Cluster purity on EN-AP 0.5091463414634146
Sample data from EN-ESSLI-2c, num of samples: 45 : "['acquire']" is assigned class exchange-exchange
exist 45 in 45
Cluster purity on EN-ESSLI-2c 0.5555555555555556
Sample data from EN-ESSLI-2b, num of samples: 40 : "['chicken']" is assigned class HI
exist 40 in 40
Cluster purity on EN-ESSLI-2b 0.8500000000000001
Sample data from EN-ESSLI-1a, num of samples: 44 : "['bottle']" is assigned class tool-artifact
exist 44 in 44
Cluster purity on EN-ESSLI-1a 0.7045454545454546
Sample data from EN-BATTIG, num of samples: 5287 : "word" is assigned class toy
exist 3442 in 5287
Cluster purity on EN-BATTIG 0.341952353282975
Sample data from EN-BLESS, num of samples: 200 : "['carp']" is assigned class water_animal
exist 193 in 200
Cluster purity on EN-BLESS 0.5181347150259068

Sample data from EN-AP, num of samples: 402 : "['acne']" is assigned class illness

Conclusion:

Utility is preserved
*   AP - goes up
*   ESSLI-2c - stays the same
*   ESSLI-2b - stays the same
*   ESSLI-1a - goes up then back down
*   BATTIG - goes up (sometimes goes back down)
*   BLESS - goes up (sometimes goes back down)






# Analogies

In [31]:
# Create text file for each category.
with open(os.path.join('data', 'analogies', 'EN-GOOGLE', 'EN-GOOGLE.txt'), "r") as file:
        lines = file.read().splitlines()

        questions = []
        answers = []
        category = []
        dummy = os.path.join('data', 'analogies', 'dummy.txt')

        f = open(dummy, "w")
        f.write("/n")
        for line in lines:
            if line.startswith(":"):
                
                # delete last "/n" and close file
                f.seek(f.tell()-1)
                f.truncate()
                f.close()

                # start new file for new category
                cat = line.lower().split()[1]
                f = open(os.path.join('data', 'analogies', f'{cat}.txt'), "w")

            else:
                f.write(line+'\n')

        os.remove(dummy)

In [34]:
def evaluate_analogies(word2vec_dict):
    # Get all word embeddings into a matrix
    vectors = []
    for word in word2vec_dict:
        vectors.append(word2vec_dict[word])
    wv = np.array(vectors)

    # Normalize embeddings
    W = np.zeros(wv.shape)
    d = (np.sum(wv ** 2, 1) ** (0.5))
    W = (wv.T / d).T

    # Create word to index dictionary
    vocab = {word:i for i, word in enumerate(word2vec_dict)}

    filenames = [
            'capital-common-countries.txt', 'capital-world.txt', 'currency.txt',
            'city-in-state.txt', 'family.txt', 'gram1-adjective-to-adverb.txt',
            'gram2-opposite.txt', 'gram3-comparative.txt', 'gram4-superlative.txt',
            'gram5-present-participle.txt', 'gram6-nationality-adjective.txt',
            'gram7-past-tense.txt', 'gram8-plural.txt', 'gram9-plural-verbs.txt',
            ]
    prefix = os.path.join('data', 'analogies')

    # to avoid memory overflow, could be increased/decreased
    # depending on system and vocab size
    split_size = 100

    correct_sem = 0; # count correct semantic questions
    correct_syn = 0; # count correct syntactic questions
    correct_tot = 0 # count correct questions
    count_sem = 0; # count all semantic questions
    count_syn = 0; # count all syntactic questions
    count_tot = 0 # count all questions
    full_count = 0 # count all questions, including those with unknown words

    for i in range(len(filenames)):
        with open('%s/%s' % (prefix, filenames[i]), 'r') as f:
            full_data = [line.rstrip().split(' ') for line in f]
            full_count += len(full_data)
            data = [x for x in full_data if all(word in vocab for word in x)]
            
        if data:
            indices = np.array([[vocab[word] for word in row] for row in data])
            ind1, ind2, ind3, ind4 = indices.T

            predictions = np.zeros((len(indices),))
            num_iter = int(np.ceil(len(indices) / float(split_size)))
            for j in range(num_iter):
                subset = np.arange(j*split_size, min((j + 1)*split_size, len(ind1)))

                pred_vec = (W[ind2[subset], :] - W[ind1[subset], :]
                    +  W[ind3[subset], :])
                #cosine similarity if input W has been normalized
                dist = np.dot(W, pred_vec.T)

                for k in range(len(subset)):
                    dist[ind1[subset[k]], k] = -np.Inf
                    dist[ind2[subset[k]], k] = -np.Inf
                    dist[ind3[subset[k]], k] = -np.Inf

                # predicted word index
                predictions[subset] = np.argmax(dist, 0).flatten()

            val = (ind4 == predictions) # correct predictions
            count_tot = count_tot + len(ind1)
            correct_tot = correct_tot + sum(val)
            if i < 5:
                count_sem = count_sem + len(ind1)
                correct_sem = correct_sem + sum(val)
            else:
                count_syn = count_syn + len(ind1)
                correct_syn = correct_syn + sum(val)

            print("%s:" % filenames[i])
            print('ACCURACY TOP1: %.2f%% (%d/%d)' %
                (np.mean(val) * 100, np.sum(val), len(val)))

    print('Questions seen/total: %.2f%% (%d/%d)' %
        (100 * count_tot / float(full_count), count_tot, full_count))
    print('Semantic accuracy: %.2f%%  (%i/%i)' %
        (100 * correct_sem / float(count_sem), correct_sem, count_sem))
    print('Syntactic accuracy: %.2f%%  (%i/%i)' %
        (100 * correct_syn / float(count_syn), correct_syn, count_syn))
    print('Total accuracy: %.2f%%  (%i/%i)' % (100 * correct_tot / float(count_tot), correct_tot, count_tot))

In [35]:
print('-'*10, 'Biased', '-'*10)
evaluate_analogies(word_vectors)
print()
print('-'*10, 'Hard Biased', '-'*10)
evaluate_analogies(new_vocab)
print()
print('-'*10, 'Double Hard Biased', '-'*10)
evaluate_analogies(db)

---------- Biased ----------
family.txt:
ACCURACY TOP1: 70.26% (267/380)
gram1-adjective-to-adverb.txt:
ACCURACY TOP1: 8.06% (75/930)
gram2-opposite.txt:
ACCURACY TOP1: 17.86% (135/756)
gram3-comparative.txt:
ACCURACY TOP1: 68.62% (914/1332)
gram4-superlative.txt:
ACCURACY TOP1: 40.91% (432/1056)
gram5-present-participle.txt:
ACCURACY TOP1: 64.68% (683/1056)
gram7-past-tense.txt:
ACCURACY TOP1: 48.85% (762/1560)
gram8-plural.txt:
ACCURACY TOP1: 33.65% (424/1260)
gram9-plural-verbs.txt:
ACCURACY TOP1: 56.14% (320/570)
Questions seen/total: 46.25% (8900/19244)
Semantic accuracy: 70.26%  (267/380)
Syntactic accuracy: 43.96%  (3745/8520)
Total accuracy: 45.08%  (4012/8900)

---------- Hard Biased ----------
family.txt:
ACCURACY TOP1: 70.26% (267/380)
gram1-adjective-to-adverb.txt:
ACCURACY TOP1: 8.06% (75/930)
gram2-opposite.txt:
ACCURACY TOP1: 17.86% (135/756)
gram3-comparative.txt:
ACCURACY TOP1: 68.62% (914/1332)
gram4-superlative.txt:
ACCURACY TOP1: 40.91% (432/1056)
gram5-present-part

Control:

Embeddings given to us by the Black is to Criminal Paper.

In [36]:
evaluate_analogies(biased_word_vectors)
evaluate_analogies(debiased_word_vectors)

family.txt:
ACCURACY TOP1: 70.26% (267/380)
gram1-adjective-to-adverb.txt:
ACCURACY TOP1: 8.06% (75/930)
gram2-opposite.txt:
ACCURACY TOP1: 17.86% (135/756)
gram3-comparative.txt:
ACCURACY TOP1: 68.62% (914/1332)
gram4-superlative.txt:
ACCURACY TOP1: 40.91% (432/1056)
gram5-present-participle.txt:
ACCURACY TOP1: 64.68% (683/1056)
gram7-past-tense.txt:
ACCURACY TOP1: 48.85% (762/1560)
gram8-plural.txt:
ACCURACY TOP1: 33.65% (424/1260)
gram9-plural-verbs.txt:
ACCURACY TOP1: 56.14% (320/570)
Questions seen/total: 46.25% (8900/19244)
Semantic accuracy: 70.26%  (267/380)
Syntactic accuracy: 43.96%  (3745/8520)
Total accuracy: 45.08%  (4012/8900)
family.txt:
ACCURACY TOP1: 72.37% (275/380)
gram1-adjective-to-adverb.txt:
ACCURACY TOP1: 8.92% (83/930)
gram2-opposite.txt:
ACCURACY TOP1: 19.31% (146/756)
gram3-comparative.txt:
ACCURACY TOP1: 68.17% (908/1332)
gram4-superlative.txt:
ACCURACY TOP1: 40.91% (432/1056)
gram5-present-participle.txt:
ACCURACY TOP1: 63.07% (666/1056)
gram7-past-tense.tx